# <font color = '#7f88ffff'> Final Project

#  <font color = '#7FFFD4'> **Parfait  TOLEFO**


### Import necessary libraries

In [3]:

# import packages
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import sqlite3
from pathlib import Path
from numpy import  nan


# url 1 : https://dakar-auto.com/senegal/voitures-4

### A function to get a container (car) data (information)

In [ ]:
def get_car_info(car):
    """
        This function 'get_car_info' is design to take as parameter a bs4 object (a car container) and return the necessary informations about this one.
        
        Usage: get_car_info(car)
    """
    try:
     
        # scrape the car brand, model and the year
        general_information_1 = car.find('h2', 'listing-card__header__title mb-md-2 mb-0').a.text.strip().split()

        # get the brand
        brand = general_information_1[0]

        # get the year
        year= general_information_1[-1]

        # Scrape the reference, kilometer driven, fuel type and gearbox type
        general_information_2  = car.find_all('li', 'listing-card__attribute list-inline-item')


        # get kilometer driven
        kilometerage = general_information_2[1].text.replace(' km','').strip()

        # get fuel type
        fuel_type = general_information_2[3].text.strip()

        # get gearbox
        gearbox = general_information_2[2].text.strip()

        # scrape the price
        price = "".join(car.find('h3', "listing-card__header__price font-weight-bold text-uppercase mb-0").text.strip().split()).replace('FCFA', '')


        # get the address
        address = ''.join(car.find('div', 'col-12 entry-zone-address').text.strip().split('\n'))

        # get the woner

        owner  = ' '.join(car.find('p','time-author m-0').text.strip().split()[1::])

    except:
        pass
    
    return (brand,year,price,address,kilometerage,gearbox,fuel_type,owner)

### A functions to get all cars informations on a page

In [ ]:
def get_all_cars(page_link):
    
    """
        This function is design to return a DataFrame of all car on a given page link. It take in parameter
        a link to a specific page.
        
        Usage: get_all_cars(page_link) 
    """
    
    # A list of the cars
    cars_list = []
    
    # A page content 
    respons = get(page_link)

    # stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
    soup = bs(respons.content,'html.parser')

    # Get all cars
    containers = soup.find_all('div','listings-cards__list-item mb-md-3 mb-3')

    for car in containers:
        try:
            car_info = get_car_info(car)
            car_dic = {
                'Brand':car_info[0],
                'Year':car_info[1],
                'Price':car_info[2],
                'Address':car_info[3],
                'Kilometerage':car_info[4],
                'Gearbox':car_info[5],
                "Fuel":car_info[6],
                "Owner":car_info[7]
                }
            cars_list.append(car_dic)
        except:
            pass
    
    return pd.DataFrame(cars_list)
        

### A function to get all page request by user

In [ ]:
def get_all_cars_all_pages(page_number):
    
    """
        This function is a principal function use to get all car information aver multiple pages.
        It use the both precedent functions to achive its goal.
        
        Usage: get_all_cars_all_pages(number_of_page)
    """
    
    df = pd.DataFrame()
    for i in range(1,page_number+1):
        url = f'https://dakar-auto.com/senegal/voitures-4?&page={i}'
        df = pd.concat([df,get_all_cars(url)],axis=0).reset_index(drop=True)
        df = df.drop_duplicates()
    return df

In [277]:
df = get_all_cars_all_pages(3)

In [ ]:
df

,Brand,Year,Price,Address,Kilometerage,Gearbox,Fuel,Owner
0,Hyundai,2018,8500,"Mbour,Thiès",165000,Automatique,Essence,Ibou SARR
1,Ford,2018,18500000,"Keur Massar,Dakar",200000,Manuelle,Diesel,Papa Gueye
2,Toyota,2023,17500000,"Keur Massar,Dakar",105000,Automatique,Essence,Papa Gueye
3,Mitsubishi,2017,16000000,"Keur Massar,Dakar",200000,Automatique,Diesel,Papa Gueye
4,Hyundai,2018,12200000,"Keur Massar,Dakar",114000,Automatique,Diesel,Papa Gueye
5,Citroen,2017,6200000,"Keur Massar,Dakar",200000,Manuelle,Diesel,Papa Gueye
6,Volkswagen,2018,19000000,"Point-E,Dakar",107000,Automatique,Essence,Ahmed Dieng
7,Hyundai,2015,5000000,"Point-E,Dakar",25000,Automatique,Essence,Sow Rassoul
8,Mitsubishi,2016,6300000,"Grand-Yoff,Dakar",1,Manuelle,Diesel,Rose DIOMPY
9,Mitsubishi,2020,9950000,"Sud Foire,Dakar",1,Manuelle,Diesel,Rose DIOMPY


# url 2: https://dakar-auto.com/senegal/motos-and-scooters-3


### A function to get a motor informations

In [ ]:
def get_motor_info(motor):
    
    """
    The function get_motor_info() is design to return the necessary information on a motor. It tahe in parameter a bs4 object (a container of a motor).
    """
    try:
            #Get brand, model, year in general info
            gen_inf_1 = motor.find('h2','listing-card__header__title mb-md-2 mb-0').text.strip().split(' ')
          
            #get the brand
            brand = gen_inf_1[0]
          
            # get the year
            year = gen_inf_1[-1]
          
            # price
            price = ''.join(motor.find('h3','listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace(' F CFA','').split())

            # address
            address =' '.join(motor.find('div','col-12 entry-zone-address').text.strip().split('\n'))

            #kilometerage
            gen_inf_2 = motor.find_all('li','listing-card__attribute list-inline-item')
            if(len(gen_inf_2)>1): # to avoid error raised by motor that doesn't have driven km, the value will be Nan (missing value)
                kilometerage  = gen_inf_2[1].text.strip().split()[0]
            else:
                kilometerage = 0 # We define missing kilometerage value to 0

            # get the owner
            owner = ' '.join(motor.find('p','time-author m-0').text.strip().split(' ')[1::])
    except:
        pass
    
    return (brand,year,price,address,kilometerage,owner)
    

### A function to get all motors on a page

In [ ]:
def get_all_motors(page_link):
    
    """
        This function, get_all_motors() take in parameter alink of a specific page and return a dataframe of all motor present on this page. 
        It use the precedent function in a for loop.
    """
    response = get(page_link)
    soup = bs(response.content,'html.parser')
    motors = soup.find_all('div','listing-card__content__inner')
    
    motors_list = []
    
    for motor in motors:
        
        motor_info = get_motor_info(motor)
        motor_dic = {
            'Brand':motor_info[0],
            'Year':motor_info[1],
            'Price':motor_info[2],
            'Address':motor_info[3],
            'Kilometerage':motor_info[4],
            'Owner':motor_info[5]
        }
        motors_list.append(motor_dic)
        
    return pd.DataFrame(motors_list)

### A function to get all motors a cros all pages requested by a user

In [ ]:
def get_all_motors_all_pages(page_number):
    
    """
        This function get_all_motors_all_pages() take in parameter the number of page upiwant to scrape 
        and return a DataFrame contain all motors across those pages. It use the two precedent functions.
        Usage: get_all_motors_all_pages(3)
    """
    # An empty data frame to stock by adding the dataframes for each page
    all_motors_df = pd.DataFrame() 
         
    for page in range(1,page_number+1):
        
        # a full url with a spacific page number 
        url = f'https://dakar-auto.com/senegal/motos-and-scooters-3?&page={page}'
        
        response = get(url)
        soup = bs(response.content,'html.parser')
        motors = soup.find_all('div','listing-card__content__inner')
        all_motors_df = pd.concat([all_motors_df,get_all_motors(url)],axis=0).reset_index(drop=True)
        
    return all_motors_df.dropna().drop_duplicates()

In [289]:
df2 = get_all_motors_all_pages(3)
df2

,Brand,Year,Price,Address,Kilometerage,Owner
0,SYM,2023,620000,"Rufisque, Dakar",1200,Lamine Ndao
1,Yamaha,2023,4300000,"VDN, Dakar",1,Rose DIOMPY
2,Yamaha,2025,800000,"Sicap Baobab, Dakar",250,Rose DIOMPY
3,Honda,2021,810000,"Cambérène, Dakar",350,Rose DIOMPY
4,KTM,2025,220000,"VDN, Dakar",6000,Malick konte
5,Yamaha,2025,950000,"Sicap Sacré Coeur, Dakar",1,Lune Mané
6,Yamaha,2017,1800000,"Saint-Louis, Saint-Louis",39000,Abdou Latif Sy
7,Haouju,2024,580000,"Médina, Dakar",2777,Franck Dieudonné DJIKOLDINGAM
8,Suzuki,2023,600000,"Mermoz Sacré-Coeur, Dakar",200,Nd Wn
9,Yamaha,2017,2150000,"Ouest Foire, Dakar",50,Birahim Ndiaye


### url 3: https://dakar-auto.com/senegal/location-de-voitures-19

### A function to get an hire car informations

In [ ]:
def get_hire_car_info(hire_car):
    
    """
    This function help to get the cessary information about a care to hire. It take in parameter a bs4 object, the container of a care. Those informations are returned in tuple to avoid accidentel modification.
    
    Usage: get_hire_car_info(hire_car_soup)
    """
    
    try:
        
        # get the brand
        brand = hire_car.find('h2','listing-card__header__title mb-md-2 mb-0').text.strip().split()[0]

        # get year
        year = hire_car.find('h2','listing-card__header__title mb-md-2 mb-0').text.strip().split()[-1]

        # get the price 
        price = "".join(hire_car.find('h3', "listing-card__header__price font-weight-bold text-uppercase mb-0").text.strip().split()).replace('FCFA', '')

        # get the address
        address = ''.join(hire_car.find('div', 'col-12 entry-zone-address').text.strip().split('\n'))

        # get the Owner
        owner  = ' '.join(hire_car.find('p','time-author m-0').text.strip().split()[1::])
                
    except:
        pass
    
    return (brand,year,price,address,owner)

### A function to get all hire car on a page

In [ ]:
def get_all_hire_cars(page_link):
    
    
    """
        This function help you to get all hire car on a specific given page via the page link or url. It return a DataFrame contain all the informations.
        
        Usage: get_all_hire_cars(page_link)
    """
    
    # get the code 
    response = get(page_link)
            
    # store the code in a Beautifulsoup objet 
    soup = bs(response.content,'html.parser')

    # find the containers  (hiring car)
    hire_cars = soup.find_all("div","listing-card__content p-2")
    
    hire_car_list = []
    
    for hire_car in hire_cars:
        hire_car_info = get_hire_car_info(hire_car)
        
        hire_car_dic = {
            'Brand':hire_car_info[0],
            'Year':hire_car_info[1],
            'Price':hire_car_info[2],
            'Address':hire_car_info[3],
            'Owner':hire_car_info[4]
        }
        hire_car_list.append(hire_car_dic)
        
    return pd.DataFrame(hire_car_list)

### A function to get all hire car cross all pages requested

In [ ]:
def get_all_hire_cars_all_page(number_pages):
    
    """
        This function return all information about all hire car on the first number of page you given in parameter.
        Usege: get_all_hire_cars_all_page(number_of_page)
    """
    
    # the base url
    base_url = 'https://dakar-auto.com/senegal/location-de-voitures-19?&page={}'
    
    # Data for all hire care
    hire_car_df = pd.DataFrame()
    
    for page in range(1,number_pages+1):
    
        url = base_url.format(page)  # buld a url for specic page
        tmp_df = get_all_hire_cars(url) # get the content of the page
        hire_car_df = pd.concat([hire_car_df,tmp_df],axis=0).reset_index(drop=True) #concat the information a page and the pasts pages
        hire_car_df = hire_car_df.dropna().drop_duplicates() # drop the missing values and the duplicated rows
    return hire_car_df

In [294]:
df3 = get_all_hire_cars_all_page(3)
df3

,Brand,Year,Price,Address,Owner
0,Ford,2013,30000,"Guédiawaye,Dakar",TERANGUA BII
1,Ford,2013,4500000,"Guédiawaye,Dakar",TERANGUA BII
2,Ford,2017,45000,"Guédiawaye,Dakar",Mouhamed Sene
3,Jeep,2016,60000,"Cambérène,Dakar",Mouhamed Sene
4,Ford,2014,30000,"Médina,Dakar",Mouhamed Sene
5,Hyundai,2015,30000,"Parcelles Assainies,Dakar",Mouhamed Sene
6,Ford,2016,30000,"Parcelles Assainies,Dakar",Gallas FAYE
7,Mazda,2016,50000,"Ouest Foire,Dakar",Mouhamed Sene
8,Citroen,2014,20000,"Médina,Dakar",Chasseur Automobile
9,Peugeot,2013,30000,"Grand-Yoff,Dakar",Chic Drive


In [5]:



def save_in_db(data_state, csv_files):
    
    data_state = data_state.lower().strip()

    if data_state == "clean":
        db_name = "cleaned_data.db"
    else:
        db_name = "uncleaned_data.db"

    # nonexion to the corresponding data base
    conn = sqlite3.connect(db_name)

    try:
        for csv_path in csv_files:
            path_obj = Path(csv_path)

            # We take the cvs file name withou the extension as the corresponding table
            table_name = path_obj.stem  
            
            # reading of a csv file
            df = pd.read_csv(path_obj)

            # writing the dataframe read in the database
            df.to_sql(table_name, conn, if_exists="replace", index=False)

    finally:
        conn.close()

In [27]:
save_in_db("clean",["Dakar_Auto_Hire_Auto.csv","Dakar_Auto_Sale.csv","Daka_Auto_Motors_Scooters.csv"])

### Cleaning the data get with web scraper

#### 1- Cleaning of Dakar_Auto_Hire_Auto.csv

In [8]:
Dakar_Auto_Hire_Auto = pd.read_csv('Dakar_Auto_Hire_Auto.csv')

tmp_df = Dakar_Auto_Hire_Auto.copy()
Dakar_Auto_Hire_Auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  138 non-null    int64 
 1   Brand       138 non-null    object
 2   Year        138 non-null    int64 
 3   Address     138 non-null    object
 4   Owner       138 non-null    object
 5   Price       138 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 6.6+ KB


In [41]:
Dakar_Auto_Hire_Auto.isna().sum()

web_scraper_order        0
web_scraper_start_url    0
Car_list                 0
Brand                    0
Year                     0
Price                    0
Address                  0
Owner                    0
dtype: int64

In [42]:
print(Dakar_Auto_Hire_Auto.duplicated().sum())

0


In [43]:
# Resign the brand withou the description
Dakar_Auto_Hire_Auto["Brand"] = tmp_df["Brand"].str.split(' ').str[0]


In [44]:
# Resign the y year withou the description
Dakar_Auto_Hire_Auto["Year"] = tmp_df["Brand"].str.split(' ').str[-2]

In [ ]:
# drop useless columns
Dakar_Auto_Hire_Auto = Dakar_Auto_Hire_Auto.drop(
    columns=['web_scraper_order', 'web_scraper_start_url', 'Car_list']
)


In [170]:
# suite of instruction to converte  the price in int64 type and fill missing value by the mode
price= tmp_df["Price"].str.split(' ').str[0]
Dakar_Auto_Hire_Auto["Price"] = price
Dakar_Auto_Hire_Auto["Price"] = Dakar_Auto_Hire_Auto["Price"].replace("NC", nan)
Dakar_Auto_Hire_Auto["Price"] = Dakar_Auto_Hire_Auto["Price"].str.replace(" ", "").str.replace("\u202f", "")
Dakar_Auto_Hire_Auto["Price"] = pd.to_numeric(Dakar_Auto_Hire_Auto["Price"], errors="coerce")
mode_value = Dakar_Auto_Hire_Auto["Price"].mode()[0]
Dakar_Auto_Hire_Auto["Price"] = Dakar_Auto_Hire_Auto["Price"].fillna(mode_value)
Dakar_Auto_Hire_Auto["Price"] = Dakar_Auto_Hire_Auto["Price"].astype(int)


In [320]:
print(Dakar_Auto_Hire_Auto.isna().sum())
print("Duplicates: ",Dakar_Auto_Hire_Auto.duplicated().sum())


Brand      0
Year       0
Address    0
Owner      0
Price      0
dtype: int64
Duplicates:  0


In [ ]:
# Save back in Dakar_Auto_Hire_Auto.csv
Dakar_Auto_Hire_Auto = Dakar_Auto_Hire_Auto.drop_duplicates()
Dakar_Auto_Hire_Auto.head()
Dakar_Auto_Hire_Auto.to_csv('Dakar_Auto_Hire_Auto.csv',index=False)

In [28]:
Dakar_Auto_Hire_Auto.head(2)

,Brand,Year,Address,Owner,Price
0,Ford,2013,Sicap Liberté,KINGCARS Service,30000
1,Ford,2012,Sicap Liberté,KINGCARS Service,40000


#### 2- Cleaning of Dakar_Auto_Sale.csv

In [15]:
Dakar_Auto_Sale = pd.read_csv('Dakar_Auto_Sale.csv')
tmp_df = Dakar_Auto_Sale.copy()

In [217]:
Dakar_Auto_Sale= tmp_df.copy()

In [218]:
Dakar_Auto_Sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3940 entries, 0 to 3939
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web_scraper_order      3940 non-null   object
 1   web_scraper_start_url  3940 non-null   object
 2   car_list               3940 non-null   object
 3   Brand                  3940 non-null   object
 4   Year                   3940 non-null   object
 5   Price                  3940 non-null   object
 6   Address                3940 non-null   object
 7   Kilometerage           3940 non-null   object
 8   Gearbox                3925 non-null   object
 9   Fuel                   3318 non-null   object
 10  Owner                  3940 non-null   object
dtypes: object(11)
memory usage: 338.7+ KB


In [219]:
# drop useless columns
Dakar_Auto_Sale = Dakar_Auto_Sale.drop(
    columns=['web_scraper_order', 'web_scraper_start_url', 'car_list']
)

In [221]:
# Resign the brand withou the description
Dakar_Auto_Sale["Brand"] = tmp_df["Brand"].str.split(' ').str[0]

In [222]:
# Resign the brand withou the description
Dakar_Auto_Sale["Kilometerage"] = tmp_df["Kilometerage"].str.split(' ').str[0]

In [223]:
# Correct year column
Dakar_Auto_Sale["Year"] = tmp_df["Year"].str.split(' ').str[1]

In [224]:
# suite of instruction to converte  the price in int64 type and fill missing value by the mode
price= tmp_df["Price"].str.split(' ').str[0]
Dakar_Auto_Sale["Price"] = price
Dakar_Auto_Sale["Price"] = Dakar_Auto_Sale["Price"].replace("NC", nan)
Dakar_Auto_Sale["Price"] = Dakar_Auto_Sale["Price"].str.replace(" ", "").str.replace("\u202f", "")
Dakar_Auto_Sale["Price"] = pd.to_numeric(Dakar_Auto_Sale["Price"], errors="coerce")
mode_value = Dakar_Auto_Sale["Price"].mode()[0]
Dakar_Auto_Sale["Price"] = Dakar_Auto_Sale["Price"].fillna(mode_value)
Dakar_Auto_Sale["Price"] = Dakar_Auto_Sale["Price"].astype(int)

In [225]:
Dakar_Auto_Sale.isna().sum()

Brand             0
Year            622
Price             0
Address           0
Kilometerage      0
Gearbox          15
Fuel            622
Owner             0
dtype: int64

In [226]:
# Fill missing value by the mode
Dakar_Auto_Sale["Year"] = Dakar_Auto_Sale["Year"].fillna(Dakar_Auto_Sale["Year"].mode()[0])

Dakar_Auto_Sale["Fuel"] = Dakar_Auto_Sale["Fuel"].fillna(Dakar_Auto_Sale["Fuel"].mode()[0])

Dakar_Auto_Sale["Gearbox"] = Dakar_Auto_Sale["Gearbox"].fillna(Dakar_Auto_Sale["Gearbox"].mode()[0])
 

In [ ]:
# Clean the Kilometerage by replacing missing values by int(mean)
for i in range(len(Dakar_Auto_Sale["Kilometerage"])):
    try:
        Dakar_Auto_Sale.loc[i,"Kilometerage"] = int(Dakar_Auto_Sale.loc[i]["Kilometerage"])
    except:
        Dakar_Auto_Sale.loc[i,"Kilometerage"] = nan
    
    
Dakar_Auto_Sale["Kilometerage"] = Dakar_Auto_Sale["Kilometerage"].fillna(int(Dakar_Auto_Sale["Kilometerage"].mean()))


In [272]:
# Cleaning Gearbox by replacing by the mode
for i in range(len(Dakar_Auto_Sale["Gearbox"])):
    
    if Dakar_Auto_Sale.loc[i,"Gearbox"] not in ["Manuelle","Automatique"]:
        Dakar_Auto_Sale.loc[i,"Gearbox"] = Dakar_Auto_Sale["Gearbox"].mode()[0]

In [319]:
print(Dakar_Auto_Sale.isna().sum())
print("Duplicates: ",Dakar_Auto_Sale.duplicated().sum())


Brand           0
Year            0
Price           0
Address         0
Kilometerage    0
Gearbox         0
Fuel            0
Owner           0
dtype: int64
Duplicates:  0


In [18]:
Dakar_Auto_Sale = Dakar_Auto_Sale.drop_duplicates()

# Save back to Dakar_Auto_Sale.csv
Dakar_Auto_Sale.to_csv('Dakar_Auto_Sale.csv',index=False)

In [29]:
Dakar_Auto_Sale.head(2)

,Brand,Year,Price,Address,Kilometerage,Gearbox,Fuel,Owner
0,Ford,2013,8500000,Fass,81000,Manuelle,Essence,Babacar DIEYE
1,Peugeot,2008,2450000,Fass,120000,Manuelle,Essence,Babacar DIEYE


#### 3- Cleaning of Daka_Auto_Motors_Scooters.csv

In [23]:
Daka_Auto_Motors_Scooters = pd.read_csv("Daka_Auto_Motors_Scooters.csv")
tmp_df = Daka_Auto_Motors_Scooters.copy()

In [281]:
# drop useless columns
Daka_Auto_Motors_Scooters = Daka_Auto_Motors_Scooters.drop(
    columns=['web_scraper_order', 'web_scraper_start_url', 'Motors_list']
)

In [282]:
# Resign the brand withou the description
Daka_Auto_Motors_Scooters["Brand"] = tmp_df["Brand"].str.split(' ').str[0]

In [284]:
# Correct year column
Daka_Auto_Motors_Scooters["Year"] = tmp_df["Year"].str.split(' ').str[1]

In [ ]:
# Keping only the number without the unit (km)
Daka_Auto_Motors_Scooters["Kilometerage"] = Daka_Auto_Motors_Scooters["Kilometerage"].str.split().str[0]

In [296]:
# Clean the Kilometerage by replacing missing values by int(mean)

for i in range(len(Daka_Auto_Motors_Scooters["Kilometerage"])):
    try:
        Daka_Auto_Motors_Scooters.loc[i,"Kilometerage"] = int(Daka_Auto_Motors_Scooters.loc[i]["Kilometerage"])
    except:
        Daka_Auto_Motors_Scooters.loc[i,"Kilometerage"] = nan   
    
Daka_Auto_Motors_Scooters["Kilometerage"] = Daka_Auto_Motors_Scooters["Kilometerage"].fillna(int(Daka_Auto_Motors_Scooters["Kilometerage"].mode()[0]))


/tmp/ipykernel_432415/3887005261.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Daka_Auto_Motors_Scooters["Kilometerage"] = Daka_Auto_Motors_Scooters["Kilometerage"].fillna(int(Daka_Auto_Motors_Scooters["Kilometerage"].mode()[0]))


In [301]:
# Correct year column
Daka_Auto_Motors_Scooters["Year"] = tmp_df["Year"].str.split(' ').str[1]

Daka_Auto_Motors_Scooters["Year"] = Daka_Auto_Motors_Scooters["Year"].fillna(Daka_Auto_Motors_Scooters["Year"].mode()[0])

In [303]:
# suite of instruction to converte  the price in int64 type and fill missing value by the mode
price= tmp_df["Price"].str.split(' ').str[0]
Daka_Auto_Motors_Scooters["Price"] = price
Daka_Auto_Motors_Scooters["Price"] = Daka_Auto_Motors_Scooters["Price"].replace("NC", nan)
Daka_Auto_Motors_Scooters["Price"] = Daka_Auto_Motors_Scooters["Price"].str.replace(" ", "").str.replace("\u202f", "")
Daka_Auto_Motors_Scooters["Price"] = pd.to_numeric(Daka_Auto_Motors_Scooters["Price"], errors="coerce")
mode_value = Daka_Auto_Motors_Scooters["Price"].mode()[0]
Daka_Auto_Motors_Scooters["Price"] = Daka_Auto_Motors_Scooters["Price"].fillna(mode_value)
Daka_Auto_Motors_Scooters["Price"] = Daka_Auto_Motors_Scooters["Price"].astype(int)

In [311]:
print(Daka_Auto_Motors_Scooters.isna().sum())


Brand           0
Year            0
Price           0
Address         0
Kilometerage    0
Owner           0
dtype: int64


In [312]:
print(Daka_Auto_Motors_Scooters.duplicated().sum())

100


In [313]:
Daka_Auto_Motors_Scooters = Daka_Auto_Motors_Scooters.drop_duplicates()

In [26]:
Daka_Auto_Motors_Scooters.to_csv("Daka_Auto_Motors_Scooters.csv",index=False)

In [30]:
Daka_Auto_Motors_Scooters.head(2)

,Brand,Year,Price,Address,Kilometerage,Owner
0,Honda,2010,250000,Grand-Dakar,1,Parking DEMBA DIOP
1,Piaggio,2010,700000,Grand-Dakar,1,Parking KHADIM SYLLA


In [ ]:
# Import necessary libraries

import streamlit as st
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import io
import time
import sqlite3
from pathlib import Path
import matplotlib.pyplot as plt


################################## Set of function to handle scraping over  url 1 : https://dakar-auto.com/senegal/voitures-4 #############################

                            # <----------------- A function to get a container (car) data (information)   ------------------------------->
 

def get_car_info(car):
    """
        This function 'get_car_info' is design to take as parameter a bs4 object (a car container) and return the necessary informations about this one.
        Those informations are returned in tuple to avoid accidentel modification.
        
        Usage: get_car_info(car)
    """
    try:
     
        # scrape the car brand, model and the year
        general_information_1 = car.find('h2', 'listing-card__header__title mb-md-2 mb-0').a.text.strip().split()

        # get the brand
        brand = general_information_1[0]

        # get the year
        year= general_information_1[-1]

        # Scrape the reference, kilometer driven, fuel type and gearbox type
        general_information_2  = car.find_all('li', 'listing-card__attribute list-inline-item')


        # get kilometer driven
        kilometerage = general_information_2[1].text.replace(' km','').strip()

        # get fuel type
        fuel_type = general_information_2[3].text.strip()

        # get gearbox
        gearbox = general_information_2[2].text.strip()

        # scrape the price
        price = "".join(car.find('h3', "listing-card__header__price font-weight-bold text-uppercase mb-0").text.strip().split()).replace('FCFA', '')


        # get the address
        address = ''.join(car.find('div', 'col-12 entry-zone-address').text.strip().split('\n'))

        # get the woner

        owner  = ' '.join(car.find('p','time-author m-0').text.strip().split()[1::])

    except:
        pass
    
    return (brand,year,price,address,kilometerage,gearbox,fuel_type,owner)



                            # <----------------- A functions to get all cars informations on a page   ------------------------------->
                            
    
def get_all_cars(page_link):
    
    """
        This function is design to return a DataFrame of all car on a given page link. It take in parameter
        a link to a specific page.
        
        Usage: get_all_cars(page_link) 
    """
    
    # A list of the cars
    cars_list = []
    
    # A page content 
    respons = get(page_link)

    # stock the html in a beautifulsoup objet with a html parser (a parser allows to easily navigate through the html code)
    soup = bs(respons.content,'html.parser')

    # Get all cars
    containers = soup.find_all('div','listings-cards__list-item mb-md-3 mb-3')

    for car in containers:
        try:
            car_info = get_car_info(car)
            car_dic = {
                'Brand':car_info[0],
                'Year':car_info[1],
                'Price':car_info[2],
                'Address':car_info[3],
                'Kilometerage':car_info[4],
                'Gearbox':car_info[5],
                "Fuel":car_info[6],
                "Owner":car_info[7]
                }
            cars_list.append(car_dic)
        except:
            pass
    
    return pd.DataFrame(cars_list)
        

                            # <----------------- A function to get all page request by user   ------------------------------->


def get_all_cars_all_pages(page_number):
    
    """
        This function is a principal function use to get all car information aver multiple pages.
        It use the both precedent functions to achive its goal. All missing values and duplicates are drop.
        
        Usage: get_all_cars_all_pages(number_of_page)
    """
    
    df = pd.DataFrame()
    for i in range(1,page_number+1):
        url = f'https://dakar-auto.com/senegal/voitures-4?&page={i}'
        df = pd.concat([df,get_all_cars(url)],axis=0).reset_index(drop=True)
        df = df.drop_duplicates()
    return df

##############################################End of set of URL 1 Scraping functions #######################################################################









######################### Set of function to handle scraping over  url 2: https://dakar-auto.com/senegal/motos-and-scooters-3 #############################


                            # <----------------- A function to get a motor informations   ------------------------------->

                    
def get_motor_info(motor):
    
    """
    The function get_motor_info() is design to return the necessary information on a motor. It tahe in parameter a bs4 object (a container of a motor). Those informations are returned in tuple to avoid accidentel modification.
    
    Usage: get_motor_info(a_motor_soup)
    """
    try:
            #Get brand, model, year in general info
            gen_inf_1 = motor.find('h2','listing-card__header__title mb-md-2 mb-0').text.strip().split(' ')
          
            #get the brand
            brand = gen_inf_1[0]
          
            # get the year
            year = gen_inf_1[-1]
          
            # price
            price = ''.join(motor.find('h3','listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace(' F CFA','').split())

            # address
            address =' '.join(motor.find('div','col-12 entry-zone-address').text.strip().split('\n'))

            #kilometerage
            gen_inf_2 = motor.find_all('li','listing-card__attribute list-inline-item')
            if(len(gen_inf_2)>1): # to avoid error raised by motor that doesn't have driven km, the value will be Nan (missing value)
                kilometerage  = gen_inf_2[1].text.strip().split()[0]
            else:
                kilometerage = 0 # We define missing kilometerage value to 0

            # get the owner
            owner = ' '.join(motor.find('p','time-author m-0').text.strip().split(' ')[1::])
    except:
        pass
    
    return (brand,year,price,address,kilometerage,owner)
    



                            # <----------------- A function to get all motors on a page   ------------------------------->

def get_all_motors(page_link):
    
    """
        This function, get_all_motors() take in parameter alink of a specific page and return a dataframe of all motor present on this page. 
        It use the precedent function in a for loop.
    """
    response = get(page_link)
    soup = bs(response.content,'html.parser')
    motors = soup.find_all('div','listing-card__content__inner')
    
    motors_list = []
    
    for motor in motors:
        
        motor_info = get_motor_info(motor)
        motor_dic = {
            'Brand':motor_info[0],
            'Year':motor_info[1],
            'Price':motor_info[2],
            'Address':motor_info[3],
            'Kilometerage':motor_info[4],
            'Owner':motor_info[5]
        }
        motors_list.append(motor_dic)
        
    return pd.DataFrame(motors_list)


                            # <----------------- A function to get all motors a cros all pages requested by a user   ------------------------------->
                            
def get_all_motors_all_pages(page_number):
    
    """
        This function get_all_motors_all_pages() take in parameter the number of page upiwant to scrape 
        and return a DataFrame contain all motors across those pages. It use the two precedent functions. All missing values and duplicates are drop.
        Usage: get_all_motors_all_pages(3)
    """
    # An empty data frame to stock by adding the dataframes for each page
    all_motors_df = pd.DataFrame() 
         
    for page in range(1,page_number+1):
        
        # a full url with a spacific page number 
        url = f'https://dakar-auto.com/senegal/motos-and-scooters-3?&page={page}'
        
        response = get(url)
        soup = bs(response.content,'html.parser')
        motors = soup.find_all('div','listing-card__content__inner')
        all_motors_df = pd.concat([all_motors_df,get_all_motors(url)],axis=0).reset_index(drop=True)
        
    return all_motors_df.dropna().drop_duplicates()


##############################################End of set of URL 2 Scraping functions #######################################################################








######################### Set of function to handle scraping over  url 3: https://dakar-auto.com/senegal/location-de-voitures-19 #############################


                            # <----------------- A function to get an hire car informations   ------------------------------->


def get_hire_car_info(hire_car):
    
    """
    This function help to get the cessary information about a care to hire. It take in parameter a bs4 object, the container of a care. Those informations are returned in tuple to avoid accidentel modification.
    
    Usage: get_hire_car_info(hire_car_soup)
    """
    
    try:
        
        # get the brand
        brand = hire_car.find('h2','listing-card__header__title mb-md-2 mb-0').text.strip().split()[0]

        # get year
        year = hire_car.find('h2','listing-card__header__title mb-md-2 mb-0').text.strip().split()[-1]

        # get the price 
        price = "".join(hire_car.find('h3', "listing-card__header__price font-weight-bold text-uppercase mb-0").text.strip().split()).replace('FCFA', '')

        # get the address
        address = ''.join(hire_car.find('div', 'col-12 entry-zone-address').text.strip().split('\n'))

        # get the Owner
        owner  = ' '.join(hire_car.find('p','time-author m-0').text.strip().split()[1::])
                
    except:
        pass
    
    return (brand,year,price,address,owner)






                            # <----------------- A function to get all hire car on a page   ------------------------------->
                        
def get_all_hire_cars(page_link):
    
    
    """
        This function help you to get all hire car on a specific given page via the page link or url. It return a DataFrame contain all the informations.
        
        Usage: get_all_hire_cars(page_link)
    """
    
    # get the code 
    response = get(page_link)
            
    # store the code in a Beautifulsoup objet 
    soup = bs(response.content,'html.parser')

    # find the containers  (hiring car)
    hire_cars = soup.find_all("div","listing-card__content p-2")
    
    hire_car_list = []
    
    for hire_car in hire_cars:
        hire_car_info = get_hire_car_info(hire_car)
        
        hire_car_dic = {
            'Brand':hire_car_info[0],
            'Year':hire_car_info[1],
            'Price':hire_car_info[2],
            'Address':hire_car_info[3],
            'Owner':hire_car_info[4]
        }
        hire_car_list.append(hire_car_dic)
        
    return pd.DataFrame(hire_car_list)




                            # <----------------- A function to get all hire car cross all pages requested   ------------------------------->

def get_all_hire_cars_all_page(number_pages):
    
    """
        This function return all information about all hire car on the first number of page you given in parameter. All missing values and duplicates are drop.
        Usege: get_all_hire_cars_all_page(number_of_page)
    """
    
    # the base url
    base_url = 'https://dakar-auto.com/senegal/location-de-voitures-19?&page={}'
    
    # Data for all hire care
    hire_car_df = pd.DataFrame()
    
    for page in range(1,number_pages+1):
    
        url = base_url.format(page)  # buld a url for specic page
        tmp_df = get_all_hire_cars(url) # get the content of the page
        hire_car_df = pd.concat([hire_car_df,tmp_df],axis=0).reset_index(drop=True) #concat the information a page and the pasts pages
        hire_car_df = hire_car_df.dropna().drop_duplicates() # drop the missing values and the duplicated rows
    return hire_car_df


##############################################End of set of URL 3 Scraping functions #######################################################################


############################################## Function to save scraped data (with web scraper in sql db) #######################################################################

def save_in_db(data_state, csv_files):
    
    data_state = data_state.lower().strip()

    if data_state == "clean":
        db_name = "cleaned_data.db"
    else:
        db_name = "uncleaned_data.db"

    # nonexion to the corresponding data base
    conn = sqlite3.connect(db_name)

    try:
        for csv_path in csv_files:
            path_obj = Path(csv_path)

            # We take the cvs file name withou the extension as the corresponding table
            table_name = path_obj.stem  
            
            # reading of a csv file
            df = pd.read_csv(path_obj)

            # writing the dataframe read in the database
            df.to_sql(table_name, conn, if_exists="replace", index=False)

    finally:
        conn.close()
 
 
# This line was run once
#save_in_db("unclean",["Dakar_Auto_Hire_Auto.csv","Dakar_Auto_Sale.csv","Daka_Auto_Motors_Scooters.csv"])
  
############################################## Function to save scraped data (with web scraper in sql db) #######################################################################



# ================== CONFIG GENERALE ==================
st.set_page_config(page_title="Scraping App", layout="wide")




def run_scraper(scrap_type: str, n_pages: int) -> pd.DataFrame:
    if scrap_type == "Car for sale":
        return get_all_cars_all_pages(n_pages)
    elif scrap_type == "Motor for sale":
        return get_all_motors_all_pages(n_pages)
    elif scrap_type == "Car for hire":
        return get_all_hire_cars_all_page(n_pages)
    else:
        raise ValueError(f"Unknown type: {scrap_type}")


# ================== FILE CONVERTER ==================
def convert_to_files(df: pd.DataFrame):
    csv_bytes = df.to_csv(index=False).encode("utf-8")

    xlsx_buffer = io.BytesIO()
    with pd.ExcelWriter(xlsx_buffer, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name="Data")
    xlsx_buffer.seek(0)

    return csv_bytes, xlsx_buffer


# ================== DATABASE LOADER (UNCLEANED) ==================
TABLE_MAPPING = {
    "Car for sale": "Dakar_Auto_Sale",
    "Motor for sale": "Daka_Auto_Motors_Scooters",
    "Car for hire": "Dakar_Auto_Hire_Auto",
}

def load_unclean_table(data_type: str, db_path="uncleaned_data.db") -> pd.DataFrame:
    """Load the corresponding table from SQLite DB."""
    table_name = TABLE_MAPPING[data_type]

    if not Path(db_path).exists():
        raise FileNotFoundError(f"Database file '{db_path}' does not exist.")

    conn = sqlite3.connect(db_path)
    try:
        df = pd.read_sql(f'SELECT * FROM "{table_name}"', conn)
    finally:
        conn.close()

    return df


def load_clean_table(data_type: str, db_path="cleaned_data.db") -> pd.DataFrame:
    """Load the corresponding cleaned table from SQLite DB."""
    table_name = TABLE_MAPPING[data_type]

    if not Path(db_path).exists():
        raise FileNotFoundError(f"Database file '{db_path}' does not exist.")

    conn = sqlite3.connect(db_path)
    try:
        df = pd.read_sql(f'SELECT * FROM "{table_name}"', conn)
    finally:
        conn.close()

    return df

# ================== SIDEBAR (MENUS IMBRIQUÉS) ==================
with st.sidebar:
    st.title("📌 Menu")

    # --- 1) SCRAPING DATA ---
    with st.expander("🔎 Scraping data", expanded=True):

        scrap_type = st.radio(
            "Choose data to scrape",
            ["Car for sale", "Motor for sale", "Car for hire"],
            key="scrape_type"
        )

        n_pages = st.number_input(
            "Number of pages",
            min_value=1,
            max_value=1000,
            value=1,
            step=1,
            key="scrape_pages"
        )

        scrape_btn = st.button("Validate", key="scrape_btn")

    # --- 2) DOWNLOAD SCRAPED DATA ---
    with st.expander("📥 Download scraped data", expanded=False):
        st.write(
            "Select the dataset you want to download. "
            "The data will be loaded directly from the local database (`uncleaned_data.db`)."
        )

        download_type = st.radio(
            "Dataset:",
            ["Car for sale", "Motor for sale", "Car for hire"],
            key="download_type"
        )

        download_btn = st.button("Load data", key="download_btn")

    # --- 3) VIEW DASHBOARD ---
    with st.expander("📊 View dashboard of data", expanded=False):
        st.write(
            "Select the cleaned dataset you want to explore. "
            "The charts are generated from the `cleaned_data.db` database."
        )

        dashboard_type = st.radio(
            "Dataset:",
            ["Car for sale", "Motor for sale", "Car for hire"],
            key="dash_type"
        )

        show_dashboard = st.button("Show dashboard", key="dash_btn")



    # --- 4) EVALUATE US ---
    with st.expander("Evaluate us", expanded=False):
        st.write("Help us improve this app by filling in a short evaluation form.")

        eval_choice = st.radio(
            "Choose an evaluation form:",
            ["Google Form", "Kobo form"],
            key="eval_choice"
        )

        eval_btn = st.button("Open selected form", key="eval_btn")


# ================== ZONE PRINCIPALE (DROITE) ==================
st.title("The first luxury app for scraping")


# --------- CASE 1 : SCRAPING STARTED ---------
if scrape_btn:
    st.subheader(f"Scraping: {scrap_type}")

    progress = st.progress(0)
    status = st.empty()

    status.info("Scraping is running...")

    # Fake progress bar for UX
    for pct in range(0, 100, 10):
        progress.progress(pct)
        time.sleep(0.1)

    try:
        df = run_scraper(scrap_type, int(n_pages))

        progress.progress(100)
        status.success("Scraping completed successfully 🎉")

        st.write("### Results preview")
        st.dataframe(df, use_container_width=True)

        csv_bytes, xlsx_buffer = convert_to_files(df)

        st.download_button(
            "Download CSV",
            data=csv_bytes,
            file_name=f"{scrap_type.replace(' ', '_').lower()}.csv",
            mime="text/csv"
        )

        st.download_button(
            "Download XLSX",
            data=xlsx_buffer,
            file_name=f"{scrap_type.replace(' ', '_').lower()}.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )

    except Exception as e:
        progress.empty()
        status.error(f"Error while scraping: {e}")


# --------- CASE 2 : DOWNLOAD / LOAD FROM DB ---------
elif download_btn:
    st.subheader(f"Unclean data loaded from database: {download_type}")

    try:
        df = load_unclean_table(download_type)
    except Exception as e:
        st.error(f"Error loading data: {e}")
    else:
        if df.empty:
            st.warning("The table is empty in the database.")
        else:
            st.write("### Data preview")
            st.dataframe(df, use_container_width=True)

            csv_bytes, xlsx_buffer = convert_to_files(df)

            st.download_button(
                "Download CSV",
                data=csv_bytes,
                file_name=f"{download_type.replace(' ', '_').lower()}_download.csv",
                mime="text/csv"
            )

            st.download_button(
                "Download XLSX",
                data=xlsx_buffer,
                file_name=f"{download_type.replace(' ', '_').lower()}_download.xlsx",
                mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            )


# --------- CASE 3 : DASHBOARD ---------
elif show_dashboard:
    st.subheader(f"Dashboard – {dashboard_type}")

    try:
        df = load_clean_table(dashboard_type)
    except Exception as e:
        st.error(f"Error loading cleaned data: {e}")
    else:
        if df.empty:
            st.warning("The cleaned table is empty in the database.")
        else:
            # Nettoyage rapide des types
            if "Price" in df.columns:
                df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

            if "Year" in df.columns:
                df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

            st.write("### Data preview")
            st.dataframe(df.head(20), use_container_width=True)

            # ===== 1) Distribution des prix =====
            if "Price" in df.columns:
                price_series = df["Price"].dropna()

                if not price_series.empty:
                    fig1, ax1 = plt.subplots()
                    ax1.hist(price_series, bins=30)
                    ax1.set_title("Price distribution")
                    ax1.set_xlabel("Price")
                    ax1.set_ylabel("Count")
                    st.pyplot(fig1)
                else:
                    st.info("No valid price data available for the histogram.")

            # ===== 2) Prix moyen par marque (Top 10) =====
            if "Price" in df.columns and "Brand" in df.columns:
                brand_price = (
                    df.dropna(subset=["Price"])
                      .groupby("Brand")["Price"]
                      .mean()
                      .sort_values(ascending=False)
                      .head(10)
                )

                if not brand_price.empty:
                    fig2, ax2 = plt.subplots()
                    brand_price.plot(kind="bar", ax=ax2)
                    ax2.set_title("Average price by brand (Top 10)")
                    ax2.set_xlabel("Brand")
                    ax2.set_ylabel("Average price")
                    plt.xticks(rotation=45, ha="right")
                    st.pyplot(fig2)
                else:
                    st.info("Not enough data to compute average price by brand.")

            # ===== 3) Nombre de véhicules par année =====
            if "Year" in df.columns:
                year_counts = (
                    df["Year"].dropna()
                      .astype(int)
                      .value_counts()
                      .sort_index()
                )

                if not year_counts.empty:
                    fig3, ax3 = plt.subplots()
                    year_counts.plot(kind="bar", ax=ax3)
                    ax3.set_title("Number of vehicles by year")
                    ax3.set_xlabel("Year")
                    ax3.set_ylabel("Count")
                    plt.xticks(rotation=45, ha="right")
                    st.pyplot(fig3)
                else:
                    st.info("No valid year data available to plot counts by year.")


# --------- CASE 4 : EVALUATION (FORM SELECTION) ---------
elif eval_btn:
    if eval_choice == "Google Form":
        st.subheader("📝 Evaluation form (Google Forms)")

        st.markdown(
            "[Open this form in a new tab]"
            "(https://docs.google.com/forms/d/e/1FAIpQLSfuOpE215IKGv9_dblMhAHUWbsksVSIm4Bxo8brheqWpJIDhA/viewform?usp=publish-editor)"
        )

        st.markdown(
            """
            <iframe 
                src="https://docs.google.com/forms/d/e/1FAIpQLSfuOpE215IKGv9_dblMhAHUWbsksVSIm4Bxo8brheqWpJIDhA/viewform?embedded=true" 
                width="100%" 
                height="700" 
                frameborder="0" 
                marginheight="0" 
                marginwidth="0">
            Loading…
            </iframe>
            """,
            unsafe_allow_html=True
        )

    else:
        st.subheader("📝 Evaluation form (KoboToolbox)")

        st.markdown(
            "[Open this form in a new tab](https://ee.kobotoolbox.org/x/7sT3NBO8)"
        )

        st.markdown(
            """
            <iframe 
                src="https://ee.kobotoolbox.org/x/7sT3NBO8" 
                width="100%" 
                height="700" 
                frameborder="0" 
                marginheight="0" 
                marginwidth="0">
            Loading…
            </iframe>
            """,
            unsafe_allow_html=True
        )


# --------- DEFAULT ---------
else:
    st.info("Use the menu on the left to start scraping, download data, view the dashboard, or evaluate the app.")
